In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 46.0 MB/s eta 0:00:00


H vs E vs S

EXP 6 : H (Z) vs E (N) vs S
No optimizer and No scaler

In [6]:
# Import necessary libraries
import os
import numpy as np
import pywt  # PyWavelets for wavelet transforms
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)
            data.append(file_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Apply DB4 wavelet transform at level 5 to each signal
def apply_wavelet_transform(data, wavelet='db4', level=5):
    transformed_data = []
    for signal in data:
        # Apply wavelet transform (db4) at level 5
        coeffs = pywt.wavedec(signal.flatten(), wavelet, level=level)
        # Use only the approximation coefficients (you can adjust if you need details)
        approximation_coeffs = coeffs[0]
        transformed_data.append(approximation_coeffs)

    return np.array(transformed_data)

# Paths to your datasets
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_N, labels_N = load_data_from_directory(path_N, label=1)  # Epileptic (N)
data_S, labels_S = load_data_from_directory(path_S, label=2)  # Seizure (S)

# Combine data and labels
if data_Z.size > 0 and data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_N, labels_S), axis=0)

    # Apply wavelet transform to the data (DB4 at level 5)
    data_all_transformed = apply_wavelet_transform(data_all, wavelet='db4', level=5)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all_transformed, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for classification
    y_train = to_categorical(y_train, num_classes=3)
    y_test = to_categorical(y_test, num_classes=3)

    # Define the ANN model (without optimizer)
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 output classes
    ])

    # Compile the model with default optimizer (no explicit optimizer)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and show training progress
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Print evaluation metrics
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Confusion Matrix: \n", conf_matrix)
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")




       # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp6/No_Options.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3323 - loss: 227.6159 - val_accuracy: 0.6333 - val_loss: 56.8374
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6666 - loss: 25.7117 - val_accuracy: 0.5444 - val_loss: 26.2701
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9101 - loss: 3.0025 - val_accuracy: 0.5667 - val_loss: 22.9572
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9038 - loss: 3.5196 - val_accuracy: 0.6111 - val_loss: 19.3330
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9434 - loss: 3.1931 - val_accuracy: 0.5222 - val_loss: 30.5713
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7812 - loss: 6.6273 - val_accuracy: 0.5111 - val_loss: 28.3771
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9329 - loss: 3.5897 - val_accuracy: 0.5444 - val_loss: 27.1847
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9401 - loss: 2.7232 - val_accuracy: 0.4778 -

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

Final Evaluation Metrics:
Test Accuracy:  0.6111111111111112
Test Recall:  0.6144620811287478
Test Precision:  0.6095860566448802
Confusion Matrix: 
 [[21  8  6]
 [ 8 12  8]
 [ 5  0 22]]
Model saved successfully at /content/drive/MyDrive/Models/Walvet_dec/ANN/Exp6/No_Options.keras ✅


EXP 6 with StandardScaler

In [7]:
import os
import time
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)
            data.append(file_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Define a function to apply wavelet decomposition (DB4, level 5)
def wavelet_decomposition(data, wavelet='db4', level=5):
    decomposed_data = []
    for sample in data:
        # Perform wavelet decomposition on each sample
        coeffs = pywt.wavedec(sample.flatten(), wavelet, level=level)
        # Reconstruct the signal using the approximation coefficients (you can change this part as needed)
        reconstructed_signal = np.concatenate(coeffs)  # Flattening and concatenating the coefficients
        decomposed_data.append(reconstructed_signal)
    return np.array(decomposed_data)

# Paths to your datasets
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_N, labels_N = load_data_from_directory(path_N, label=1)  # Epileptic (N)
data_S, labels_S = load_data_from_directory(path_S, label=2)  # Seizure (S)

# Combine data and labels
if data_Z.size > 0 and data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_N, labels_S), axis=0)

    # Apply wavelet decomposition to the data before flattening
    data_all = wavelet_decomposition(data_all, wavelet='db4', level=5)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Scale the data using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)  # Fit and transform training data
    X_test = scaler.transform(X_test)  # Only transform testing data

    # One-hot encode the labels for classification
    y_train = to_categorical(y_train, num_classes=3)
    y_test = to_categorical(y_test, num_classes=3)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 output classes
    ])

    # Compile the model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    # Track training time
    start_training_time = time.time()

    # Train the model and show training progress
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    end_training_time = time.time()
    training_duration = end_training_time - start_training_time

    # Evaluate the model on the test set
    start_evaluation_time = time.time()

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    end_evaluation_time = time.time()
    evaluation_duration = end_evaluation_time - start_evaluation_time

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Print evaluation metrics and timings
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Confusion Matrix: \n", conf_matrix)
    print("\nTiming Information:")
    print(f"Training Duration: {training_duration:.2f} seconds")
    print(f"Evaluation Duration: {evaluation_duration:.2f} seconds")
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


           # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp6/With_Stander.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.4130 - loss: 1.1342 - val_accuracy: 0.6889 - val_loss: 0.6635
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9753 - loss: 0.2290 - val_accuracy: 0.7111 - val_loss: 0.9073
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9951 - loss: 0.0423 - val_accuracy: 0.6556 - val_loss: 1.1398
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0096 - val_accuracy: 0.6667 - val_loss: 1.3105
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 0.6556 - val_loss: 1.4711
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 0.6444 - val_loss: 1.5696
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.6222 - val_loss: 1.6747
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 7.2870e-04 - val_accuracy: 0.6111 - val_loss: 1.7739
Epoch 9

with op

In [4]:
import os
import time
import numpy as np
import pywt  # PyWavelets for wavelet transforms
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)
            data.append(file_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Apply DB4 wavelet transform at level 5 to each signal
def apply_wavelet_transform(data, wavelet='db4', level=5):
    transformed_data = []
    for signal in data:
        # Apply wavelet transform (db4) at level 5
        coeffs = pywt.wavedec(signal.flatten(), wavelet, level=level)
        # Use only the approximation coefficients (you can adjust if you need details)
        approximation_coeffs = coeffs[0]
        transformed_data.append(approximation_coeffs)

    return np.array(transformed_data)

# Paths to your datasets
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_N, labels_N = load_data_from_directory(path_N, label=1)  # Epileptic (N)
data_S, labels_S = load_data_from_directory(path_S, label=2)  # Seizure (S)

# Combine data and labels
if data_Z.size > 0 and data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_N, labels_S), axis=0)

    # Apply wavelet transform to the data (DB4 at level 5)
    data_all_transformed = apply_wavelet_transform(data_all, wavelet='db4', level=5)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all_transformed, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for classification
    y_train = to_categorical(y_train, num_classes=3)
    y_test = to_categorical(y_test, num_classes=3)

    # Define the ANN model
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Input layer specifying input shape
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 output classes
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Track training time
    start_training_time = time.time()

    # Train the model and show training progress
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    end_training_time = time.time()
    training_duration = end_training_time - start_training_time

    # Evaluate the model on the test set
    start_evaluation_time = time.time()

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    end_evaluation_time = time.time()
    evaluation_duration = end_evaluation_time - start_evaluation_time

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Print evaluation metrics and timings
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Confusion Matrix: \n", conf_matrix)
    print("\nTiming Information:")
    print(f"Training Duration: {training_duration:.2f} seconds")
    print(f"Evaluation Duration: {evaluation_duration:.2f} seconds")
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


   # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp6/With_Optimiser.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3181 - loss: 108.5432 - val_accuracy: 0.4889 - val_loss: 47.1896
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6707 - loss: 20.0502 - val_accuracy: 0.6222 - val_loss: 19.1427
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7973 - loss: 6.1585 - val_accuracy: 0.5556 - val_loss: 27.0303
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9018 - loss: 2.8447 - val_accuracy: 0.6111 - val_loss: 22.1856
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9390 - loss: 0.9417 - val_accuracy: 0.6778 - val_loss: 12.0115
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9830 - loss: 1.3986 - val_accuracy: 0.6111 - val_loss: 17.0505
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9498 - loss: 1.0427 - val_accuracy: 0.5889 - val_loss: 26.4524
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9262 - loss: 1.6289 - val_accuracy: 0.6111 - val_los

EXP 6 : with Standerscaler and Optimiser Adam

In [3]:
import os
import time
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)
            data.append(file_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Define a function to apply wavelet decomposition (DB4, level 5)
def wavelet_decomposition(data, wavelet='db4', level=5):
    decomposed_data = []
    for sample in data:
        # Perform wavelet decomposition on each sample
        coeffs = pywt.wavedec(sample.flatten(), wavelet, level=level)
        # Reconstruct the signal using the approximation coefficients (you can change this part as needed)
        reconstructed_signal = np.concatenate(coeffs)  # Flattening and concatenating the coefficients
        decomposed_data.append(reconstructed_signal)
    return np.array(decomposed_data)

# Paths to your datasets
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_N = '/content/drive/My Drive/PFA2_DataSet/DataSet/Elipy/N'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_N, labels_N = load_data_from_directory(path_N, label=1)  # Epileptic (N)
data_S, labels_S = load_data_from_directory(path_S, label=2)  # Seizure (S)

# Combine data and labels
if data_Z.size > 0 and data_N.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_N, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_N, labels_S), axis=0)

    # Apply wavelet decomposition to the data before flattening
    data_all = wavelet_decomposition(data_all, wavelet='db4', level=5)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # Scale the data using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)  # Fit and transform training data
    X_test = scaler.transform(X_test)  # Only transform testing data

    # One-hot encode the labels for classification
    y_train = to_categorical(y_train, num_classes=3)
    y_test = to_categorical(y_test, num_classes=3)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 output classes
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Track training time
    start_training_time = time.time()

    # Train the model and show training progress
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    end_training_time = time.time()
    training_duration = end_training_time - start_training_time

    # Evaluate the model on the test set
    start_evaluation_time = time.time()

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    end_evaluation_time = time.time()
    evaluation_duration = end_evaluation_time - start_evaluation_time

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Print evaluation metrics and timings
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Confusion Matrix: \n", conf_matrix)
    print("\nTiming Information:")
    print(f"Training Duration: {training_duration:.2f} seconds")
    print(f"Evaluation Duration: {evaluation_duration:.2f} seconds")
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


           # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp6/With_Options.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.4609 - loss: 0.9195 - val_accuracy: 0.5222 - val_loss: 0.8743
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9874 - loss: 0.1975 - val_accuracy: 0.6000 - val_loss: 1.1965
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.0347 - val_accuracy: 0.6222 - val_loss: 1.6756
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 0.6111 - val_loss: 2.1287
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 5.7191e-04 - val_accuracy: 0.6111 - val_loss: 2.4437
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 1.6333e-04 - val_accuracy: 0.6000 - val_loss: 2.6352
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 1.0581e-04 - val_accuracy: 0.5889 - val_loss: 2.7342
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 7.0810e-05 - val_accuracy: 0.5889 - val_loss: 2.